In [0]:
!pip install -qqq python-dotenv
!pip install -qqq llama_index.core
!pip install -qqq nest_asyncio


In [0]:
import nest_asyncio
from dotenv import load_dotenv
from llama_index.core import SimpleDirectoryReader
_ = load_dotenv()
nest_asyncio.apply()

In [0]:
documents = SimpleDirectoryReader(input_files=["meta-gpt-paper.pdf"]).load_data()

In [0]:
from llama_index.core.node_parser import SentenceSplitter

spliter  = SentenceSplitter(chunk_size=1024)
nodes = spliter.get_nodes_from_documents(documents)

In [0]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model='gpt-3.5-turbo')
Settings.embed_model = OpenAIEmbedding(model = 'text-embedding-ada-002')

In [0]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [0]:
summar_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True
)
vector_query_engine = vector_index.as_query_engine()

In [0]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summar_query_engine,
    description=("Useful for summarization questions related to MetaGPT")
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=("Useful for retrieving specific context from MetaGPT paper")
)

In [0]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector = LLMSingleSelector.from_defaults(),
    query_engine_tools= [summary_tool, vector_tool],
    verbose=True
)

In [0]:
nest_asyncio.apply()
response = query_engine.query("What is the summary of the document?")
print (str(response))

In [0]:
response.response

In [0]:

print(len(response.source_nodes))

In [0]:
response = query_engine.query("How do Agents share information with other agents?")

In [0]:
response.response